In [96]:
!java -version

openjdk version "11.0.19" 2023-04-18
OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)


In [97]:
!pip install tabula-py
!pip install requests

In [98]:
import tabula
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [99]:
tabula.environment_info()

Python version:
    3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0]
Java version:
    openjdk version "11.0.19" 2023-04-18
OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)
tabula-py version: 2.7.0
platform: Linux-5.15.109+-x86_64-with-glibc2.35
uname:
    uname_result(system='Linux', node='5f3508751db4', release='5.15.109+', version='#1 SMP Fri Jun 9 10:57:30 UTC 2023', machine='x86_64')
linux_distribution: ('Ubuntu', '22.04', 'jammy')
mac_ver: ('', ('', '', ''), '')


In [100]:
##test
pdf_path = 'https://www.fdma.go.jp/disaster/heatstroke/items/r5/heatstroke_sokuhouti_20230710.pdf'
dfs = tabula.read_pdf(pdf_path, stream=True, pages=1)

In [101]:
df = dfs[0]
df_base = df.iloc[2:7, [0, 5]]

In [102]:
##Auto test

In [103]:
url = 'https://www.fdma.go.jp/disaster/heatstroke/post3.html'
r = requests.get(url)
text = r.text
soup = BeautifulSoup(text)

div_tag = soup.find_all('div', {'class': 'link-wrap mb30 txt'})

href_list = []
for i in div_tag[0].find_all('a', class_='link'):
  href_tag = i.get('href')
  href_list.append(href_tag)

href_list_app = []
for i in range(len(href_list)):
    test = list(href_list[i])
    test[:0] = 'https://www.fdma.go.jp/'
    test = ''.join(test)
    href_list_app.append(test)

href_list_app.reverse()

In [104]:
dfs = tabula.read_pdf(href_list_app[0], stream=True, pages=1)
df = dfs[0]
df_base = df.iloc[2:9, [0, 5]]

In [105]:
for pdf_path in href_list_app[1:]:
  dfs = tabula.read_pdf(pdf_path, stream=True, pages=1)
  df = dfs[0]
  df_add = df.iloc[2:9, [0, 5]]
  df_base = pd.concat([df_base, df_add], axis=0)

In [106]:
df_base = df_base.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 4': '熱中症患者数'})

In [107]:
df_base.to_csv('hot_weahter.csv')